In [1]:
import numpy as np
import pandas as pd

In [2]:
places = pd.read_csv('./kyoto-geocoded-3.csv')

In [3]:
from datetime import datetime as dt

def hm2minutes(str: str):
    diff = dt.strptime(str, '%H:%M') - dt.strptime('0:00', '%H:%M')
    return diff.seconds / 60

In [4]:
hm2minutes('11:00'), hm2minutes('14:00')

(660.0, 840.0)

In [5]:
lunch_candidates = ['東福寺三門', '安養寺木造阿弥陀如来像', '建仁寺方丈', '嵐山通船', ]
lunch_places = places[places['Name'].isin(lunch_candidates)].copy()

In [6]:
lunch_places

,ID,Name,Address,lat,lng,Unnamed: 5,Unnamed: 6,open,close,open_time_min,close_time_min,Unnamed: 11,time_to_spend
4,1000017,東福寺三門,本町15丁目778,34.977097,135.774106,900,1600,9:00:00,16:00:00,540,960,20.0,20
7,1000043,建仁寺方丈,東山区大和大路通四条下る４丁目小松町584番地,35.000455,135.773639,1000,1600,10:00:00,16:00:00,600,960,NaN,33
12,1000151,安養寺木造阿弥陀如来像,中京区新京極通蛸薬師東南角東側町511,35.005769,135.767334,1000,1600,10:00:00,16:00:00,600,960,10.0,10
81,8000001,嵐山通船,南乗船場・京都市西京区嵐山山中町/北乗船場・京都市右京区嵯峨天竜寺芒ノ馬場町,35.009449,135.666773,900,1700,9:00:00,17:00:00,540,1020,30.0,30


In [7]:
lunch_places['Name'] = lunch_places['Name'].apply(lambda x: x + '_lunch')
lunch_places['time_to_spend'] = 60
lunch_places['open_time_min'] = hm2minutes('11:00')
lunch_places['close_time_min'] = hm2minutes('14:00')
places = pd.concat([places, lunch_places], axis=0)

In [8]:
places

,ID,Name,Address,lat,lng,Unnamed: 5,Unnamed: 6,open,close,open_time_min,close_time_min,Unnamed: 11,time_to_spend
0,9999999,京都駅,京都市下京区東塩小路釜殿町,34.980983,135.754836,530,2359,5:30:00,23:59:00,330.0,1439.0,0.0,0
1,1000002,慈照寺（銀閣寺）銀閣,京都市左京区銀閣寺町2,35.026864,135.798269,830,1700,8:30:00,17:00:00,510.0,1020.0,40.0,40
2,1000003,銀閣寺（慈照寺）東求堂,京都市左京区銀閣寺町2,35.026864,135.798269,830,1700,8:30:00,17:00:00,510.0,1020.0,40.0,40
3,1000010,下鴨神社（賀茂御祖神社）東本殿西本殿,京都市左京区下鴨泉川町59,35.034883,135.771905,630,1700,6:30:00,17:00:00,390.0,1020.0,30.0,30
4,1000017,東福寺三門,本町15丁目778,34.977097,135.774106,900,1600,9:00:00,16:00:00,540.0,960.0,20.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,8000026,嵯峨野トロッコ列車,京都市右京区嵯峨天龍寺車道町,35.018158,135.680484,900,1730,9:00:00,17:30:00,540.0,1050.0,25.0,25
4,1000017,東福寺三門_lunch,本町15丁目778,34.977097,135.774106,900,1600,9:00:00,16:00:00,660.0,840.0,20.0,60
7,1000043,建仁寺方丈_lunch,東山区大和大路通四条下る４丁目小松町584番地,35.000455,135.773639,1000,1600,10:00:00,16:00:00,660.0,840.0,NaN,60
12,1000151,安養寺木造阿弥陀如来像_lunch,中京区新京極通蛸薬師東南角東側町511,35.005769,135.767334,1000,1600,10:00:00,16:00:00,660.0,840.0,10.0,60


In [9]:
time_to_spend = places.set_index('Name')['time_to_spend'].to_dict()
open_time_min = places.set_index('Name')['open_time_min'].to_dict()
close_time_min = places.set_index('Name')['close_time_min'].to_dict()
name_list = places['Name'].to_list()

In [10]:
def manhattan_dist(lat1, lng1, lat2, lng2):
    dist_x = np.abs(lng1 - lng2) * 91287.7885
    dist_y = np.abs(lat1 - lat2) * 110940.5844
    return dist_x + dist_y

In [11]:
matrix = dict()
for f_idx, f_row in places.iterrows():
    for t_idx, t_row in places.iterrows():
        distance = manhattan_dist(f_row.lat, f_row.lng, t_row.lat, t_row.lng)
        matrix[(f_row['Name'], t_row['Name'])] = (np.ceil(15 * distance / 1000))

In [12]:
class NodeManager:
    def __init__(self, names):
        self.names = names
        self.n = len(self.names)
        self.nodes = list(range(self.n))

        self.name_map_node = dict(zip(self.names, self.nodes))
        self.node_map_name = {v: k for k, v in self.name_map_node.items()}

    def node_to_name(self, node):
        return self.node_map_name[node]

    def name_to_node(self, name):
        return self.name_map_node[name]

In [13]:
node_manager = NodeManager(name_list)

In [14]:
from ortools.constraint_solver import pywrapcp

index_manager = pywrapcp.RoutingIndexManager(node_manager.n, 1, node_manager.name_to_node('京都駅'))

In [15]:
routing = pywrapcp.RoutingModel(index_manager)


In [16]:
lunch_place_indices = [
    index_manager.NodeToIndex(node_manager.name_to_node(i))
    for i in [pl + '_lunch' for pl in lunch_candidates]
]
lunch_place_indices

[84, 86, 85, 87]

In [17]:
condition = (
    routing.ActiveVar(lunch_place_indices[0])
    + routing.ActiveVar(lunch_place_indices[1])
    + routing.ActiveVar(lunch_place_indices[2]) == 1)
routing.solver().AddConstraint(condition)

In [18]:
def time_callback(from_index, to_index):
    from_node = index_manager.IndexToNode(from_index)
    to_node = index_manager.IndexToNode(to_index)
    from_name = node_manager.node_to_name(from_node)
    to_name = node_manager.node_to_name(to_node)

    if to_name == '京都駅':
        return 0
    return matrix[from_name, to_name] + time_to_spend[from_name]

In [19]:
transit_callback_index = routing.RegisterTransitCallback(time_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [20]:
for i in node_manager.names:
    if i == '京都駅':
        continue
    index = index_manager.NodeToIndex(node_manager.name_to_node(i))
    routing.AddDisjunction([index], 10_000)

In [21]:
routing.AddDimension(
    transit_callback_index,
    60,
    60 * 20,
    False,
    'Time'
)
time_dimension = routing.GetDimensionOrDie('Time')

In [22]:
for i in node_manager.names:
    index = index_manager.NodeToIndex(node_manager.name_to_node(i))
    open_time = 330.0 if np.isnan(open_time_min[i]) else open_time_min[i]
    close_time = 1439.0 if np.isnan(close_time_min[i]) else close_time_min[i]
    # print(open_time, close_time)
    time_dimension.CumulVar(index).SetRange(int(open_time), int(close_time))

In [27]:
from ortools.constraint_solver import routing_enums_pb2

search_params = pywrapcp.DefaultRoutingSearchParameters()
search_params.time_limit.seconds = 10
search_params.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_params.log_search = True

In [28]:
solution = routing.SolveWithParameters(search_params)

In [25]:
import math

def minutes2hm(n):
    hour = math.floor(n / 60)
    min = n % 60
    return "{0}:{1:02d}".format(hour, min)

def print_solution(routing, solution):
    vehicle_id = 0
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        previous_index = index
        name = node_manager.node_to_name(index_manager.IndexToNode(previous_index))
        output = name + ': '
        output += minutes2hm(solution.Value(time_dimension.CumulVar(index)))
        print(output)
        index = solution.Value(routing.NextVar(index))
    print('京都駅:', minutes2hm(solution.Value(time_dimension.CumulVar(index))))

print_solution(routing, solution)

京都駅: 5:30
北野天満宮東門の椋: 7:18
北野天満宮影向松: 7:51
平野神社の草花と樹木: 8:29
地蔵院（椿寺）五色八重散椿: 9:16
廬山寺木造如意輪観音半跏像: 10:20
廬山寺慈恵大師自筆遺告: 10:30
廬山寺絹本著色普賢十羅刹女像: 10:40
安養寺木造阿弥陀如来像: 11:23
南岩倉明王院不動寺空海作「石像不動明王」: 11:47
蓮光寺駒止地蔵菩薩像: 12:01
蓮光寺阿弥陀如来像: 12:11
建仁寺方丈: 12:45
建仁寺方丈_lunch: 13:18
知恩院濡髪大明神: 14:42
知恩院七不思議: 14:52
知恩院紫雲水: 15:25
天授庵絹本著色細川幽斎像同夫人像: 16:16
南禅寺方丈: 16:46
京都駅: 16:46


In [26]:
lats = places.set_index('Name')['lat'].to_dict()
lngs = places.set_index('Name')['lng'].to_dict()

def solution_to_csv(routing, solution):
    vehicle_id = 0
    index = routing.Start(vehicle_id)
    result = []
    while not routing.IsEnd(index):
        previous_index = index
        name = node_manager.node_to_name(index_manager.IndexToNode(previous_index))
        time = minutes2hm(solution.Value(time_dimension.CumulVar(index)))
        lat = lats[name]
        lng = lngs[name]
        result.append([name, time, lat, lng])

        index = solution.Value(routing.NextVar(index))
    return pd.DataFrame(result, columns=['Name', 'Time', 'Lat', 'Lng'], index='Name')

res = solution_to_csv(routing, solution)

TypeError: Index(...) must be called with a collection of some kind, 'Name' was passed

In [ ]:
res


In [ ]:
res.to_csv('./kyoto-route-2.csv', index_label = ["ID"])


